In [ ]:
#Prepare all our necessary libraries
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

#pytorch libraries
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Pad, Compose,CenterCrop, ToPILImage, Normalize, ConvertImageDtype, Resize

from torchvision.models import resnet50
from torch import nn
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.optim import SGD, Adam
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_

import datetime

# libs für AE

!pip install cleverhans --upgrade
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent


# PREPARE DATA

In [ ]:
#Prepare our Dataset Structure, as it has to be normalized for pytorch

from torchvision.io import read_image

class GTSRB(Dataset):
    def __init__(self, annotations_file, img_dir , transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[["Path","ClassId"]]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
def float_transform(tensor):
    return tensor.float()

# creating training and test datasets; Normalize pictures them to (3,90,90) size
img_dir = "/kaggle/input/gtsrb-german-traffic-sign/"
train_file = "/kaggle/input/gtsrb-german-traffic-sign/Train.csv"
train_data = GTSRB(img_dir = img_dir, annotations_file = train_file,
                   transform = Compose([Resize((30,30)), ConvertImageDtype(torch.float32)]))


In [ ]:
#Prepare DataLoader
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))

In [ ]:
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print("Maximum: ",train_features[0].max(),"\t Minimum: ", train_features[0].min() )
img = train_features[0]
label = train_labels[0]
img = ToPILImage()(img).convert("RGB")
plt.imshow(img)
plt.show()
print(f"Label: {label}")

# CREATE MODEL 

In [ ]:
#downloading resent50 pretrained on ImageNet 
resn50 = resnet50(pretrained=True, progress = True)

#adjust resnet50 to my dataset
class r50(nn.Module):
    def __init__(self, pretrained_model):
        super(r50,self).__init__()
        self.rn50 = pretrained_model
        self.fl1 = nn.Linear(1000, 256)
        self.fl2 = nn.Linear(256,43)
        
    def forward(self, X):
        X = self.rn50(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X

# TRAINING

In [ ]:
model = r50(resn50)
criterion = nn.CrossEntropyLoss(reduction = "mean")
optim = Adam(model.parameters(), lr = 0.001)

#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

nb_epochs = 10

ae_predictions = []
ae_accuracy = []
accuracy = []

eps = 0.2
eps_iter = 0.01
steps = 30

for epoch in range(nb_epochs):
    running_loss = 0
    ae_count = 0
    for i, data in enumerate(train_dataloader, 0):
        train_features, train_labels = data
        train_features, train_labels = train_features.to(device), train_labels.to(device)
        
        #ae training
        model.eval()
            
        ae = projected_gradient_descent(model,train_features[-15:], eps, eps_iter, steps,np.inf, clip_min=0, clip_max=1).detach()
        
        if i == 100:
            image = ToPILImage()(ae[0]).convert("RGB")
            original = ToPILImage()(train_features[-15]).convert("RGB")
            
            fig, axs = plt.subplots(2, 1, constrained_layout=True)
            axs[0].imshow(image)
            axs[0].set_title('Adversarial example')
            fig.suptitle('Comparison AE vs. Original', fontsize=12)

            axs[1].imshow(original)
            axs[1].set_title('Original')
            plt.show()
            print("Maximum pixel distance: ", torch.max(train_features[-15] - ae[0]))
        
        
        train_features = torch.cat((train_features[:-15], ae)).to(device)
        model.train()
        #ae_count += 1
        
        #standard training
        optim.zero_grad()
        prediction = model(train_features).to(torch.float32)
        loss = criterion(prediction, train_labels.to(torch.long)) 
        loss.backward()
        optim.step()

        running_loss += loss.item()
        
        #calculate accuracy
        corrects = 0

        ae_corrects = 0
        pred = torch.argmax(prediction, dim = 1)
        
        for i, p,l in zip(range(64), pred, train_labels):
            if i > (64-15):
                ae_predictions.append((p,l))
                ae_corrects += (p==l)
                ae_count +=1
            corrects += (p == l)
        ae_accuracy.append(int(ae_corrects)/15)
        accuracy.append(int(corrects)/len(pred))


    
    
    print("Epoch: {j}/{total_epochs} \t Time: {time} \t Loss: {Loss} \t Accuracy: {acc}".format(j = epoch+1,Loss = running_loss/len(train_dataloader),total_epochs = nb_epochs,time = datetime.datetime.now().time(), acc=np.mean(accuracy[-50:])))
    print("{number} adversarial examples have been created. The accuracy against ae's is {ae_acc}".format(number=ae_count, ae_acc = np.mean(ae_accuracy[-50:])))    





In [ ]:
#save accuracy information 
accuracy_data = pd.DataFrame({"accuracy":accuracy, "ae_accuracy" : ae_accuracy})
accuracy_data.to_csv(("/kaggle/working/training_eps{}_epsiter{}_st{}".format(eps*10, eps_iter*10, steps).replace(".","") + ".csv"))

In [ ]:
# save model for our test notebook    
PATH = '/kaggle/working/gtsrbX.pth'
torch.save(model.state_dict(), PATH)

In [ ]:

#show an AE of the training set
image = ToPILImage()(ae[0]).convert("RGB")
plt.imshow(image)
plt.show()